In [1]:
from data import load, split, ScaleAbsOne
import torch
import math
from matplotlib import pyplot as plt
from sklearn.kernel_approximation import Nystroem
import numpy as np
from tqdm import tqdm
from typing import Union

In [2]:
df = load()
train, dev, test = split(df, "new_dose-response_matrices", inner_fold=1, outer_fold=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.43it/s]


In [3]:
train_target = np.array(train.pop("PercentageGrowth"))
dev_target = np.array(dev.pop("PercentageGrowth"))
test_target = np.array(test.pop("PercentageGrowth"))
scaler = ScaleAbsOne()
train = scaler.fit_transform(train.values)
dev = scaler.transform(dev.values)
test = scaler.transform(test.values)

# multi linear kernel

$$
k_\gamma(x,y) = \prod_k^K \frac{\gamma^2 + x_k y_k}{\gamma^2 + 1}
$$
alternatively use the logarithmized space
$$
k_\gamma(x, y) = \sum_k^K \log\left(\frac{\gamma^2 + x_k y_k}{\gamma^2 + 1}\right)
$$ 

In [5]:
class MultiLinearKernel:
    def __init__(self, scale: Union[float, np.array]):
        self.scale = scale
        
    def __call__(self, x1, x2):
        prod = np.einsum("nd, md -> nmd", x1, x2)
        frac = (self.scale**2 + prod) / (1 + self.scale**2)
        return frac.prod(axis=-1)

# class LogMultiLinearKernel:
#     def __init__(self, scale: int):
#         self.scale = scale  # make vector later
        
#     def __call__(self, x1, x2):
#         prod = np.einsum("nd, md -> nmd", x1, x2)
#         frac = (self.scale**2 + prod) / (1 + self.scale**2)
#         return np.log(frac).sum(axis=-1)

# Nystroem approximation

credit: https://stats.stackexchange.com/questions/261149/nystroem-method-for-kernel-approximation 

Assume $K \in R^{n\times n}$ has rank $m << n$ and let $K_{11}$ be of shape $m\times m$
\begin{align}
K = \begin{bmatrix}
K_{11} & K_{21}^T\\
K_{21} & K_{22}
\end{bmatrix}
\end{align}


Assume the eigen decomposition
\begin{align}
K &= U \Lambda U^T \\ 
&= \begin{bmatrix} U_1 \\ U_2 \end{bmatrix} \Lambda\begin{bmatrix} U_1 \\ U_2 \end{bmatrix}^T \\
&= \begin{bmatrix} U_1 \Lambda U_1^T & U_1 \Lambda U_2^T \\
U_2 \Lambda U_1^T & U_2 \Lambda U_2^T
\end{bmatrix}
\end{align}

As m << n it is easy to compute $K_{11} = U_1 \Lambda U_1^T$.

Which also allows us to solve for $U_2 = K_{21} U_1 \Lambda^{-1}$.

And substitute into 
\begin{align}
K_{22} &= U_2 \Lambda U_2^T \\
&= (K_{21} U_1 \Lambda^{-1}) \Lambda (K_{21} U_1 \Lambda^{-1})^T \\
&= K_{21} U_1 \Lambda^{-1} U_1^T K_{21}^T \\
&= K_{21} K_{11}^{-1} K_{21}^T \\
&= \left(K_{21} K_{11}^{-\frac{1}{2}}\right) \left(K_{21} K_{11}^{-\frac{1}{2}}\right)^T
\end{align}

Which implies the following explicit feature map
\begin{align}
\Phi = \begin{bmatrix}
K_{11}^{\frac{1}{2}} \\ K_{21}K_{11}^{-\frac{1}{2}}
\end{bmatrix}
\end{align}

For which we can show that 
\begin{align}
\Phi \Phi^T = 
\begin{bmatrix}
K_{11} & K_{21}^T \\
K_{21} & K_{21} K_{11}^{-1} K_{21}^T
\end{bmatrix} = K
\end{align}

We sample $m$ datapoints and compute $K_{11}^{-\frac{1}{2}}$ and arrive at the final result
$$\phi(X_\star) = K_{\star 1} K_{11}^{-\frac{1}{2}}$$

In [8]:
def batched(method):
    def wrapped(self, x):
        bs = self.batch_size
        out = [method(self, x[b:b+bs]) for b in tqdm(range(0, len(x), bs))]
        return np.concatenate(out, axis=0)
    return wrapped

class Nystroem:
    def __init__(self, kernel: callable, n_components: int, batch_size=512):
        self.n_components = n_components
        self.kernel = kernel
        self.batch_size= batch_size
        
    def fit(self, x):
        n, d = x.shape
        idx = np.random.permutation(n)[:self.n_components]
        basis = x[idx]
        
        k = self.kernel(basis, basis)
        u, s, v = np.linalg.svd(k)
        # for stability, credit sklearn
        s = np.maximum(s, 1e-12)
        
        self.normalization_ = (u / np.sqrt(s)) @ v
        self.components_ = basis
        self.component_indices_ = idx
        return self
    
    @batched
    def transform(self, x):
        embedded = self.kernel(x, self.components_)
        features = embedded @ self.normalization_.T
        return features
    
    def fit_transform(self, x):
        self.fit(x)
        return self.transform(x)

# bayesian linear regressionm

In [33]:
class BayesianLinearRegression:
    # see Bishop - Pattern Recognition And Machine Learning
    # page 31. equations (1.70 - 1.72)
    def __init__(self, alpha=1e-13, beta=1):
        self.alpha = alpha
        self.beta = beta
        
    def fit(self, x, y):
        n, d = x.shape
        S_inv = self.alpha * np.eye(d) + self.beta * x.T @ x
        self.S = np.linalg.pinv(S_inv)
        self.m = self.S @ x.T @ y
    
    def predict(self, x):
        y = self.beta * x @ self.m
        y_var = 1 / self.beta + np.einsum("ij, jk, ki -> i", x, self.S, x.T, optimize=True)
        return y, np.sqrt(y_var)
    
    def score(self, x, y):
        self.fit(x, y)
        diff = y - self.predict(x)[0]
        mse = (diff**2).mean()
        explained_variance = 1 - (diff.var() / y.var())
        return mse, explained_variance

# Comparison

In [34]:
idx = np.random.permutation(len(train))[:1024]
train_sub = train[idx]
train_target_sub = train_target[idx]

In [35]:
# plain linear regression
BayesianLinearRegression().score(train_sub, train_target_sub)

(785.4425561869423, 0.5376965167286726)

In [36]:
# polynomial kernel regression
from sklearn.kernel_approximation import PolynomialCountSketch
Xp = PolynomialCountSketch(gamma=1.0, degree=2, n_components=2048).fit_transform(train_sub)
BayesianLinearRegression().score(Xp, train_target_sub)

(2.3580375890981282e-23, 1.0)

In [37]:
# mlk-kernel regression
phi = MultiLinearKernel(scale=10)(train_sub, train_sub)
BayesianLinearRegression().score(phi, train_target_sub)

(1.7414046778352082e-08, 0.9999999999898144)

In [40]:
# mlk-nystroem regression, full
mlk = MultiLinearKernel(scale=10)
phi = Nystroem(kernel=mlk, n_components=1024).fit_transform(train_sub)
BayesianLinearRegression().score(phi, train_target_sub)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.46it/s]


(2.1395851309376377e-16, 1.0)

In [41]:
# mlk-nystroem regression half
mlk = MultiLinearKernel(scale=10)
phi = Nystroem(kernel=mlk, n_components=512).fit_transform(train_sub)
BayesianLinearRegression().score(phi, train_target_sub)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.95it/s]


(434.837760761155, 0.7440589259284915)